In [1]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
processed_data = np.load('Full_array.npy')

from tensorflow import keras

import pandas as pd

#The size of the dataset is 10,000; but I only filled the first 9166 with values, the rest are zeros.
number = 9166
processed_data = processed_data[0:number]

np.random.seed(1)
tf.set_random_seed(1)

In [2]:
#hyperparameters:
test_train_split = 0.85
batch_size = 32
epoch_number = 15
learning_rate = 0.1
dropout_rate = 0.4
CNL1_filters = 32
CNL1_kernal_size = 5
MPL1_pool_size= (2,2)
MPL1_strides = 2
CNL2_filters = 64
CNL2_kernal_size = 5
MPL2_pool_size = (2,2)
MPL2_strides = 2

#I have a non-round number of examples
train_split_indice = int(np.round(test_train_split*number))

In [3]:
#Load in my dataset of targets, targets are strings labels under the name "Class"
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv/zoo2MainSpecz.csv", usecols=[2,3,4,8], nrows=number)
Class = galaxyzoo["gz2class"].values
RA = galaxyzoo['ra'].values
DEC = galaxyzoo['dec'].values

In [4]:
#I want to take the first character of the Class string and interpret as a integer, ala MNIST example code
dictionary = {'A':int(2),'E':int(1),'S':int(0)}

In [5]:
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = int(dictionary[Class[i][0]])

In [6]:
#split my data between training and test sets
train_target = target[0:train_split_indice]
test_target = target[train_split_indice:number]
train_images = processed_data[0:train_split_indice]
test_images = processed_data[train_split_indice:number]

train_images = train_images.reshape(train_split_indice,28,28,1)
test_images = test_images.reshape(train_split_indice-number,28,28,1)

In [7]:

#define my model, using a CNN with 2 CNL layers, 2 max pool layers, 1 dense layer, 1 drop out layer, and another dense layer 
def create_model(dropout_rate):
    
    model = keras.Sequential([])
    model.add(keras.layers.Conv2D(input_shape=(28,28,1),filters=CNL1_filters,kernel_size=CNL1_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL1_pool_size, strides=MPL1_strides))
    model.add(keras.layers.Conv2D(filters=CNL2_filters,kernel_size=CNL2_kernal_size,padding="same",activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=MPL2_pool_size,strides=MPL2_strides))
    model.add(keras.layers.Reshape([7*7*64]))
    model.add(keras.layers.Dense(units=1024,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=3,activation=tf.nn.softmax))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return(model)


In [10]:
epochs_list= [5,10,15,20]
batch_size_list= [64,32,10,1]
dropout_rate_list= [0.25,0.3,0.4,0.5]

In [11]:
#cut away a validation set, needed because we will tune our model towards the thing we finally evaluate it on, so we lose
#generality
validation_target = train_target[train_split_indice-1000:train_split_indice]
validation_images = train_images[train_split_indice-1000:train_split_indice]
print(np.shape(validation_images))

(1000, 28, 28, 1)


In [12]:
train_images = train_images[0:train_split_indice-1000]
train_target = train_target[0:train_split_indice-1000]
#we have reduced our training data by 1000 to use a validation set with our improvised grid search
print(np.shape(train_images))

(6791, 28, 28, 1)


In [13]:
model = create_model(dropout_rate = 0.3) #just initialize some random weights with some random dropout
Wsave = model.get_weights()
#tuning hyperparameters: lets try to implement GridSearchCV from scikitlearn
#GridSearchCV didn't seem to run, lets just put it in a for loop
for j in range(len(batch_size_list)):
    for k in range(len(dropout_rate_list)):
        for i in range(len(epochs_list)):
            model = create_model(dropout_rate=dropout_rate_list[k])
            #reset the weights to whatever they were above
            model.set_weights(Wsave)
            model.fit(train_images, train_target, epochs=epochs_list[i], batch_size=batch_size_list[k], verbose=1, shuffle=True)
            test_loss, test_acc = model.evaluate(validation_images.reshape(1000,28,28,1), validation_target)
            print('Test accuracy:', test_acc)
            print('epoch', epochs_list[i])
            print('dropout', dropout_rate_list[k])
            print('batch size',batch_size_list[j])

Epoch 1/5
6791/6791 [==============================] - 32s 5ms/step - loss: 0.6313 - acc: 0.6972
Epoch 2/5
6791/6791 [==============================] - 30s 4ms/step - loss: 0.5423 - acc: 0.7478
Epoch 3/5
6791/6791 [==============================] - 32s 5ms/step - loss: 0.5161 - acc: 0.7662
Epoch 4/5
6791/6791 [==============================] - 33s 5ms/step - loss: 0.4938 - acc: 0.7804
Epoch 5/5
1000/1000 [==============================] - 2s 2ms/step
Test accuracy: 0.778
epoch 5
dropout 0.25
batch size 64
Epoch 1/10
6791/6791 [==============================] - 33s 5ms/step - loss: 0.6278 - acc: 0.7074
Epoch 2/10
6791/6791 [==============================] - 30s 4ms/step - loss: 0.5435 - acc: 0.7538
Epoch 3/10
6791/6791 [==============================] - 30s 4ms/step - loss: 0.5207 - acc: 0.7665
Epoch 4/10
6791/6791 [==============================] - 30s 4ms/step - loss: 0.4935 - acc: 0.7862
Epoch 5/10
6791/6791 [==============================] - 30s 4ms/step - loss: 0.4752 - acc: 0.7927

6791/6791 [==============================] - 40s 6ms/step - loss: 0.3955 - acc: 0.8226
Epoch 13/15
6791/6791 [==============================] - 40s 6ms/step - loss: 0.3814 - acc: 0.8327
Epoch 14/15
6791/6791 [==============================] - 40s 6ms/step - loss: 0.3765 - acc: 0.8332
Epoch 15/15
1000/1000 [==============================] - 2s 2ms/step
Test accuracy: 0.824
epoch 15
dropout 0.3
batch size 64
Epoch 1/20
6791/6791 [==============================] - 42s 6ms/step - loss: 0.6001 - acc: 0.7049
Epoch 2/20
6791/6791 [==============================] - 40s 6ms/step - loss: 0.5446 - acc: 0.7453
Epoch 3/20
6791/6791 [==============================] - 40s 6ms/step - loss: 0.5091 - acc: 0.7740
Epoch 4/20
6791/6791 [==============================] - 41s 6ms/step - loss: 0.4932 - acc: 0.7804
Epoch 5/20
6791/6791 [==============================] - 40s 6ms/step - loss: 0.4637 - acc: 0.7933
Epoch 6/20
6791/6791 [==============================] - 41s 6ms/step - loss: 0.4423 - acc: 0.8049
Ep

6791/6791 [==============================] - 30s 4ms/step - loss: 0.4286 - acc: 0.8131
Epoch 13/15
6791/6791 [==============================] - 30s 4ms/step - loss: 0.4029 - acc: 0.8255
Epoch 14/15
6791/6791 [==============================] - 31s 5ms/step - loss: 0.3938 - acc: 0.8282
Epoch 15/15
1000/1000 [==============================] - 2s 2ms/step
Test accuracy: 0.816
epoch 15
dropout 0.25
batch size 32
Epoch 1/20
6791/6791 [==============================] - 33s 5ms/step - loss: 0.6405 - acc: 0.6905
Epoch 2/20
6791/6791 [==============================] - 31s 5ms/step - loss: 0.5460 - acc: 0.7436
Epoch 3/20
6791/6791 [==============================] - 31s 4ms/step - loss: 0.5276 - acc: 0.7607
Epoch 4/20
6791/6791 [==============================] - 32s 5ms/step - loss: 0.4956 - acc: 0.7802
Epoch 5/20
6791/6791 [==============================] - 32s 5ms/step - loss: 0.4792 - acc: 0.7941
Epoch 6/20
6791/6791 [==============================] - 31s 5ms/step - loss: 0.4582 - acc: 0.8042
E

6791/6791 [==============================] - 582s 86ms/step - loss: 0.6088 - acc: 0.6887
Epoch 12/15
6791/6791 [==============================] - 581s 86ms/step - loss: 0.6070 - acc: 0.6911
Epoch 13/15
6791/6791 [==============================] - 581s 86ms/step - loss: 0.6030 - acc: 0.7023
Epoch 14/15
6791/6791 [==============================] - 581s 86ms/step - loss: 0.6071 - acc: 0.6987
Epoch 15/15
1000/1000 [==============================] - 3s 3ms/step
Test accuracy: 0.67
epoch 15
dropout 0.5
batch size 32
Epoch 1/20
6791/6791 [==============================] - 584s 86ms/step - loss: 0.7083 - acc: 0.5973
Epoch 2/20
6791/6791 [==============================] - 587s 86ms/step - loss: 0.7821 - acc: 0.5842
Epoch 3/20
6791/6791 [==============================] - 583s 86ms/step - loss: 0.6974 - acc: 0.5824
Epoch 4/20
6791/6791 [==============================] - 599s 88ms/step - loss: 0.6994 - acc: 0.5856
Epoch 5/20
6791/6791 [==============================] - 603s 89ms/step - loss: 4.610

6791/6791 [==============================] - 41s 6ms/step - loss: 0.5394 - acc: 0.7520
Epoch 3/5
6791/6791 [==============================] - 42s 6ms/step - loss: 0.5136 - acc: 0.7726
Epoch 4/5
6791/6791 [==============================] - 41s 6ms/step - loss: 0.4932 - acc: 0.7781: 0s - loss: 0.4933 - acc: 0.778
Epoch 5/5
1000/1000 [==============================] - 3s 3ms/step
Test accuracy: 0.803
epoch 5
dropout 0.3
batch size 10
Epoch 1/10
6791/6791 [==============================] - 44s 6ms/step - loss: 0.6117 - acc: 0.7084
Epoch 2/10
6791/6791 [==============================] - 41s 6ms/step - loss: 0.5414 - acc: 0.7467
Epoch 3/10
6791/6791 [==============================] - 41s 6ms/step - loss: 0.5137 - acc: 0.7678
Epoch 4/10
6791/6791 [==============================] - 41s 6ms/step - loss: 0.4793 - acc: 0.7922
Epoch 5/10
6791/6791 [==============================] - 41s 6ms/step - loss: 0.4657 - acc: 0.7912
Epoch 6/10
6791/6791 [==============================] - 41s 6ms/step - loss

6791/6791 [==============================] - 82s 12ms/step - loss: 0.3952 - acc: 0.8292
Epoch 14/15
6791/6791 [==============================] - 84s 12ms/step - loss: 0.3902 - acc: 0.8293
Epoch 15/15
1000/1000 [==============================] - 3s 3ms/step
Test accuracy: 0.789
epoch 15
dropout 0.4
batch size 10
Epoch 1/20
6791/6791 [==============================] - 90s 13ms/step - loss: 0.5948 - acc: 0.7102
Epoch 2/20
6791/6791 [==============================] - 82s 12ms/step - loss: 0.5492 - acc: 0.7480
Epoch 3/20
6791/6791 [==============================] - 83s 12ms/step - loss: 0.5118 - acc: 0.7653
Epoch 4/20
6791/6791 [==============================] - 83s 12ms/step - loss: 0.4997 - acc: 0.7753
Epoch 5/20
6791/6791 [==============================] - 82s 12ms/step - loss: 0.4787 - acc: 0.7862
Epoch 6/20
6791/6791 [==============================] - 82s 12ms/step - loss: 0.4682 - acc: 0.8000
Epoch 7/20
6791/6791 [==============================] - 82s 12ms/step - loss: 0.4607 - acc: 0

Epoch 2/5
6791/6791 [==============================] - 31s 5ms/step - loss: 0.5556 - acc: 0.7369
Epoch 3/5
6791/6791 [==============================] - 31s 5ms/step - loss: 0.5240 - acc: 0.7651
Epoch 4/5
6791/6791 [==============================] - 31s 5ms/step - loss: 0.5021 - acc: 0.7800
Epoch 5/5
1000/1000 [==============================] - 3s 3ms/step
Test accuracy: 0.753
epoch 5
dropout 0.25
batch size 1
Epoch 1/10
6791/6791 [==============================] - 36s 5ms/step - loss: 0.6615 - acc: 0.6856
Epoch 2/10
6791/6791 [==============================] - 32s 5ms/step - loss: 0.5573 - acc: 0.7313
Epoch 3/10
6791/6791 [==============================] - 32s 5ms/step - loss: 0.5352 - acc: 0.7569
Epoch 4/10
6791/6791 [==============================] - 32s 5ms/step - loss: 0.5092 - acc: 0.7670
Epoch 5/10
6791/6791 [==============================] - 31s 5ms/step - loss: 0.4976 - acc: 0.7825
Epoch 6/10
6791/6791 [==============================] - 32s 5ms/step - loss: 0.4750 - acc: 0.7884

6791/6791 [==============================] - 592s 87ms/step - loss: 0.5827 - acc: 0.7187
Epoch 3/5
6791/6791 [==============================] - 592s 87ms/step - loss: 0.5614 - acc: 0.7348
Epoch 4/5
6791/6791 [==============================] - 594s 88ms/step - loss: 0.5515 - acc: 0.7529
Epoch 5/5
1000/1000 [==============================] - 4s 4ms/step
Test accuracy: 0.78
epoch 5
dropout 0.5
batch size 1
Epoch 1/10
6791/6791 [==============================] - 606s 89ms/step - loss: 0.6309 - acc: 0.6834
Epoch 2/10
6791/6791 [==============================] - 594s 88ms/step - loss: 0.6000 - acc: 0.6986
Epoch 3/10
6791/6791 [==============================] - 593s 87ms/step - loss: 0.5878 - acc: 0.7210
Epoch 4/10
6791/6791 [==============================] - 596s 88ms/step - loss: 0.5731 - acc: 0.7190
Epoch 5/10
6791/6791 [==============================] - 596s 88ms/step - loss: 0.5634 - acc: 0.7310
Epoch 6/10
6791/6791 [==============================] - 642s 95ms/step - loss: 0.5605 - acc: 

In [ ]:
#evaluate what this is, I grabbed it from above, we can wait until I am doe hyperparameter tuning
test_loss, test_acc = model.evaluate(test_images.reshape(number - train_split_indice,28,28,1), test_target)
print('Test accuracy:', test
      _acc)

In [ ]:
"""
Epoch 1/15
6791/6791 [==============================] - 31s 5ms/step - loss: 0.6327 - acc: 0.7064
Epoch 2/15
6791/6791 [==============================] - 30s 4ms/step - loss: 0.5418 - acc: 0.7445
Epoch 3/15
6791/6791 [==============================] - 32s 5ms/step - loss: 0.5320 - acc: 0.7606
Epoch 4/15
6791/6791 [==============================] - 35s 5ms/step - loss: 0.5065 - acc: 0.7746
Epoch 5/15
6791/6791 [==============================] - 31s 4ms/step - loss: 0.5021 - acc: 0.7703
Epoch 6/15
6791/6791 [==============================] - 30s 4ms/step - loss: 0.4688 - acc: 0.7930
Epoch 7/15
6791/6791 [==============================] - 30s 4ms/step - loss: 0.4461 - acc: 0.8028
Epoch 8/15
6791/6791 [==============================] - 30s 4ms/step - loss: 0.4389 - acc: 0.8097
Epoch 9/15
6791/6791 [==============================] - 31s 5ms/step - loss: 0.4292 - acc: 0.8137
Epoch 10/15
6791/6791 [==============================] - 33s 5ms/step - loss: 0.4168 - acc: 0.8211
Epoch 11/15
6791/6791 [==============================] - 34s 5ms/step - loss: 0.4105 - acc: 0.8178
Epoch 12/15
6791/6791 [==============================] - 33s 5ms/step - loss: 0.3987 - acc: 0.8277
Epoch 13/15
6791/6791 [==============================] - 30s 4ms/step - loss: 0.3983 - acc: 0.8218
Epoch 14/15
6791/6791 [==============================] - 34s 5ms/step - loss: 0.3792 - acc: 0.8302
Epoch 15/15
6791/6791 [==============================] - 42s 6ms/step - loss: 0.3952 - acc: 0.8246
1000/1000 [==============================] - 3s 3ms/step
Test accuracy: 0.831
epoch 15
dropout 0.25
batch size 64



Epoch 1/20
6791/6791 [==============================] - 93s 14ms/step - loss: 0.6055 - acc: 0.7027
Epoch 2/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.5434 - acc: 0.7433
Epoch 3/20
6791/6791 [==============================] - 87s 13ms/step - loss: 0.5117 - acc: 0.7672
Epoch 4/20
6791/6791 [==============================] - 89s 13ms/step - loss: 0.4860 - acc: 0.7843
Epoch 5/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.4762 - acc: 0.7897
Epoch 6/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.4524 - acc: 0.8022
Epoch 7/20
6791/6791 [==============================] - 87s 13ms/step - loss: 0.4479 - acc: 0.8036
Epoch 8/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.4394 - acc: 0.8068
Epoch 9/20
6791/6791 [==============================] - 87s 13ms/step - loss: 0.4283 - acc: 0.8140
Epoch 10/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.4206 - acc: 0.8186
Epoch 11/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.4219 - acc: 0.8123
Epoch 12/20
6791/6791 [==============================] - 89s 13ms/step - loss: 0.4045 - acc: 0.8248
Epoch 13/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.3979 - acc: 0.8248
Epoch 14/20
6791/6791 [==============================] - 89s 13ms/step - loss: 0.3982 - acc: 0.8277
Epoch 15/20
6791/6791 [==============================] - 90s 13ms/step - loss: 0.3896 - acc: 0.8245
Epoch 16/20
6791/6791 [==============================] - 89s 13ms/step - loss: 0.3800 - acc: 0.8321
Epoch 17/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.3774 - acc: 0.8338
Epoch 18/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.3727 - acc: 0.8364
Epoch 19/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.3590 - acc: 0.8392
Epoch 20/20
6791/6791 [==============================] - 88s 13ms/step - loss: 0.3596 - acc: 0.8355
1000/1000 [==============================] - 4s 4ms/step
Test accuracy: 0.834
epoch 20
dropout 0.4
batch size 1
"""